### Missing Photo Detect

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 20000  # 28 native word end index

In [4]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [5]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [6]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [7]:
image_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/20000 Words/Images_Crop_Size"
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment"
file = "Talk Time 5 Video List.xlsx"
sheet = "Sheet1"  # Sheet1

In [8]:
df_video_file = pd.read_excel(f"{path_folder}/{file}")
df_video_file

,2Gram,3Gram,4Gram,5Gram,6Gram,7Gram
0,ve bu,ya da bir,ya da başka bir,istediğin başka bir şey var,eklemek istediğin bir şey var mı,söylemek istediğin başka bir şey var mı
1,bir soru,herhangi bir şey,bu çok büyük bir,bu benim için çok önemli,söylemek istediğin başka bir şey var,NaN
2,daha sonra,bir iki üç,kısa bir süre sonra,bu da demek oluyor ki,bir şey yok bir şey yok,NaN
3,bu konuda,çok önemli bir,iyi bir şey değil,ya da başka bir şey,eklemek istediğiniz bir şey var mı,NaN
4,bir şekilde,diye bir şey,diye bir şey yok,istediğiniz bir şey var mı,için elimden gelen her şeyi yapacağım,NaN
...,...,...,...,...,...,...
95,gerçekten çok,NaN,NaN,NaN,NaN,NaN
96,evet evet,NaN,NaN,NaN,NaN,NaN
97,evet biliyorum,NaN,NaN,NaN,NaN,NaN
98,sen ve,NaN,NaN,NaN,NaN,NaN


In [9]:
df_file_count = word_count_result(df_video_file,df_video_file.columns)
df_file_count

,word,word_count
0,bir,127
1,şey,75
2,bu,37
3,çok,33
4,var,32
...,...,...
187,hepsi,1
188,hem,1
189,hatırlıyor,1
190,göstermek,1


In [11]:
image_file_list = os.listdir(image_path_folder)
filename_without_ext_list = []
for file in image_file_list:
    # file and extention
    file_without_ext = os.path.splitext(file)[0]
    filename_without_ext_list.append(file_without_ext)
filename_without_ext_list

['ama',
 'artık',
 'asla',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'bana',
 'belki',
 'ben',
 'beni',
 'benimle',
 'bile',
 'biliyorsun',
 'biliyorum',
 'diye',
 'acele',
 'biri',
 'biz',
 'bize',
 'bizi',
 'bizim',
 'bu',
 'buna',
 'bunu',
 'burada',
 'buraya',
 'bütün',
 'gerçek',
 'böyle',
 'çok',
 'de',
 'doğru',
 'dostum',
 'dur',
 'ederim',
 'efendim',
 'et',
 'evet',
 'fazla',
 'gece',
 'gel',
 'geldi',
 'geliyor',
 'gerçekten',
 'geri',
 'gibi',
 'misin',
 'ana',
 'göre',
 'gün',
 'güzel',
 'hadi',
 'hala',
 'harika',
 'haydi',
 'her',
 'herkes',
 'hey',
 'hiçbir',
 'için',
 'ile',
 'ilk',
 'iş',
 'istiyorsun',
 'istiyorum',
 'iyi',
 'izin',
 'kadar',
 'mısın',
 'işte',
 'ki',
 'kız',
 'lazım',
 'lütfen',
 'merhaba',
 'musun',
 'anda',
 'basit',
 'nerede',
 'o',
 'olabilir',
 'olacak',
 'olan',
 'olarak',
 'oldu',
 'olduğunu',
 'olmak',
 'olmaz',
 'olsun',
 'olur',
 'oluyor',
 'ona',
 'önce',
 'onları',
 'onu',
 'onunla',
 'orada',
 'öyle',
 'pekala',
 'sadece',
 'sa

In [12]:
set_image_file = set(filename_without_ext_list)
set_video_file = set(df_file_count["word"])

In [13]:
set_video_file.difference(set_image_file)

{'adın',
 'alabilir',
 'altı',
 'andan',
 'anlıyor',
 'arada',
 'azından',
 'be',
 'beri',
 'beş',
 'demedim',
 'duyabiliyor',
 'dört',
 'ediyor',
 'eklemek',
 'elimden',
 'etmek',
 'geldiniz',
 'geldiğini',
 'geldiğiniz',
 'gelen',
 'gitti',
 'hatırlıyor',
 'herhangi',
 'hoş',
 'hoşuna',
 'ifade',
 'ihtiyacımız',
 'ilgili',
 'istediğimi',
 'itibaren',
 'işi',
 'komik',
 'konuda',
 'korkacak',
 'kısa',
 'misiniz',
 'musunuz',
 'mıydı',
 'nedir',
 'nereden',
 'olduğu',
 'olsa',
 'pek',
 'sekiz',
 'sormak',
 'söyleyebilir',
 'söz',
 'süre',
 'sürü',
 'taraftan',
 'ufak',
 'vay',
 'veriyorum',
 'yapacak',
 'yapacağım',
 'yapacağız',
 'yapıyorsunuz',
 'yedi',
 'yer',
 'yoktur',
 'zamana',
 'zamandan',
 'çocuk'}